In [1]:
import numpy as np
import pandas as pd
from sklearn import cluster, datasets, mixture
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler

In [2]:
full_table = pd.read_csv('extract_counts.tsv', sep='\t')

In [3]:
full_table.set_index(['locus_tag'], inplace=True)
full_table.head(n=1)

,product,type,gene_symbol,locus,start_coord,end_coord,note,translation,5GB1_FM69_t2_TR1,5GB1_FM69_t3_TR1,...,5GB1_FM21_TR1,5GB1_FM21_TR2,5GB1_FM21_TR2_UW,5GB1_FM03_TR1_QC,5GB1_FM03_TR2_QC,5GB1_FM20_TR1_QC,5GB1_FM20_TR2_QC,5GB1_FM20_TR3_QC,5GB1_FM21_TR1_QC,5GB1_FM21_TR2_QC
locus_tag,,,,,,,,,,,,,,,,,,,,,
METBUDRAFT_0001,hypothetical protein,CDS,kfrB,METBUDRAFT_scaffold2.1,497,844,NaN,MKQRLLVMNGQRILQGESDGAWTSQKVDKAGTLKPGIYNIYTAEPV...,71,863,...,1356,1201,1211,57,180,24,227,116,166,136


In [4]:
counts = full_table.drop(['product', 'type', 'gene_symbol', 'locus', 'start_coord', 'end_coord', 'note', 'translation'], axis=1)
counts.head(n=1)

,5GB1_FM69_t2_TR1,5GB1_FM69_t3_TR1,5GB1_FM69_t3_TR1_UW,5GB1_FM69_t4_TR1,5GB1_FM69_t4_TR1_UW,5GB1_FM80_t2_TR1,5GB1_FM80_t4_TR1,5GB1_FM81_t1_TR3,5GB1_FM81_t2_TR3,5GB1_vial_wLa_TR3,...,5GB1_FM21_TR1,5GB1_FM21_TR2,5GB1_FM21_TR2_UW,5GB1_FM03_TR1_QC,5GB1_FM03_TR2_QC,5GB1_FM20_TR1_QC,5GB1_FM20_TR2_QC,5GB1_FM20_TR3_QC,5GB1_FM21_TR1_QC,5GB1_FM21_TR2_QC
locus_tag,,,,,,,,,,,,,,,,,,,,,
METBUDRAFT_0001,71,863,88,1140,97,849,893,803,812,0,...,1356,1201,1211,57,180,24,227,116,166,136


In [5]:
pseudo_counts = counts + 1

In [6]:
reference_column = '5GB1_FM19_TR1'
reference_column = '5GB1_FM69_t2_TR1'
log2ratio = pseudo_counts.copy(deep=True)
for column in counts.columns:
    log2ratio[column] = np.log2(pseudo_counts[column] / pseudo_counts[reference_column])
log2ratio.drop([reference_column], axis=1, inplace=True)

In [7]:
log2ratio.head(n=1)

,5GB1_FM69_t3_TR1,5GB1_FM69_t3_TR1_UW,5GB1_FM69_t4_TR1,5GB1_FM69_t4_TR1_UW,5GB1_FM80_t2_TR1,5GB1_FM80_t4_TR1,5GB1_FM81_t1_TR3,5GB1_FM81_t2_TR3,5GB1_vial_wLa_TR3,5GB1_vial_woLa_TR2,...,5GB1_FM21_TR1,5GB1_FM21_TR2,5GB1_FM21_TR2_UW,5GB1_FM03_TR1_QC,5GB1_FM03_TR2_QC,5GB1_FM20_TR1_QC,5GB1_FM20_TR2_QC,5GB1_FM20_TR3_QC,5GB1_FM21_TR1_QC,5GB1_FM21_TR2_QC
locus_tag,,,,,,,,,,,,,,,,,,,,,
METBUDRAFT_0001,3.584963,0.305808,3.986158,0.444785,3.561394,3.634206,3.481127,3.497187,-6.169925,-6.169925,...,4.23628,4.061296,4.073249,-0.311944,1.329921,-1.526069,1.662965,0.70044,1.213779,0.928107


Add in the gene start and end locatio

In [17]:
# makes me nervous in case the ordering of rows has changes
# should probably be done with a pandas table join on locus_tag index
log2ratio['start_coord'] = full_table['start_coord']
log2ratio['end_coord'] = full_table['end_coord']

Use standard scaler on the data frame to prepare it for clustering

In [8]:
X = StandardScaler().fit_transform(log2ratio)

In [9]:
db = DBSCAN(eps=15., min_samples=10).fit(X)

In [10]:
labels = db.labels_
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
n_clusters_

1

In [12]:
for eps in range(1, 42):
    db = DBSCAN(eps=eps, min_samples=10).fit(X)
    labels = db.labels_
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    print(str(eps) + " " + str(n_clusters_))

1 1
2 3
3 2
4 3
5 5
6 4
7 3
8 1
9 1
10 2
11 1
12 1
13 1
14 1
15 1
16 1
17 1
18 1
19 1
20 1
21 1
22 1
23 1
24 1
25 1
26 1
27 1
28 1


KeyboardInterrupt: 

In [14]:
db = DBSCAN(eps=5, min_samples=10).fit(X)